In [ ]:
import pandas as pd
import datetime as dt
df = pd.read_csv('data/melb_data_fe.csv', sep=',')



df['Date'] = pd.to_datetime(df["Date"])
df['Quarter'] = pd.PeriodIndex(df['Date'] ,freq='Q')
#print(df.info())

#print(df.head())
#print(df.describe())
optima = df.nunique(axis=0, dropna=True)
#df["Rooms"] = df["Rooms"].astype("category")
#print(df.info())
#from collections import Counter
exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
#for i in df.columns:
#    if i not in exclude and optima[i] < 150:
#        print(i)
#        print(optima[i])
#        df[i] = df[i].astype('category')
#print(Counter(df['Quarter']).most_common()[2])
#df.sort_values(by='Date', ascending=False).head(10)
#df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

#mask1 = df['AreaRatio'] < -0.8
#mask2 = df['Type'] == 'townhouse'
#mask3 = df['SellerG'] == 'McGrath'
#df[mask1&mask2&mask3].sort_values(
#    by=['Date', 'AreaRatio'],
#    ascending=[True, False],
#    ignore_index=True
#).loc[:,['Date','AreaRatio']]

#print(df.sort_values(by='AreaRatio', ascending=False, ignore_index=True).iloc[1558:]['BuildingArea'])
#print(df.info())
mask4=df['Type']=='townhouse'
mask5=df['Rooms'] > 2
df[mask4&mask5].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
)
mask6 = df['Quarter'] == '2017Q2'
df[mask6].groupby(by=['Type','Rooms'])['Price'].sum().sort_values()
df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()


pivot = df.pivot_table(
    values='BuildingArea',
    index='Rooms',
    columns='Type',
    aggfunc=['median'],
    fill_value=0
)
#print(pivot)
#pivot.columns
#display(pivot['mean']['unit'])

#mask7 = pivot['mean']['house'] < pivot['median']['house']
#filtered_pivot = pivot[mask7]
#display(filtered_pivot)
#print(list(filtered_pivot.index))

pivot = df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='mean',
    fill_value=0
)
#print(pivot['unit'].sort_values())

In [ ]:
import pandas as pd
import datetime as dt
imdb = pd.read_csv('data/movies.csv', sep=',')
imdb.info()
imdb_users = pd.read_csv('data/ratings1.csv', sep=',')
print(imdb_users.nunique(axis=0, dropna=True))
imdb_dates = pd.read_csv('data/dates.csv', sep=',')
#imdb_dates['date'] = pd.to_datetime(imdb_dates["date"])
#imdb_dates['Year'] = imdb_dates['date'].dt.year
#imdb_dates.iloc['date'] = pd.to_datetime(imdb_dates['date'])
# add a column for Year
#imdb_dates['Year'] = imdb_dates['date'].dt.year
#imdb_dates.pivot_table(
#    values=imdb_dates[0].dt.year
#)
r_1 = pd.read_csv('data/ratings1.csv', sep=',')
r_2 = pd.read_csv('data/ratings2.csv', sep=',')

imdb_dates['date'] = pd.to_datetime(imdb_dates['date'])
imdb_dates['years'] = imdb_dates['date'].dt.year
#print(imdb_dates['years'].mode())
imdb_df = pd.concat([r_1, r_2], ignore_index=True)
#display(imdb_df)

#display(r_1.tail(1))
#display(r_2.head(1))

#print('Число строк в таблице ratings: ', imdb_df.shape[0])

imdb_df = imdb_df.drop_duplicates(ignore_index=True)

#print('Число строк в таблице ratings: ', imdb_df.shape[0])

imdb_df = pd.concat([imdb_df, imdb_dates], axis=1)
#display(imdb_df.tail(10))

movies = pd.read_csv('data/movies.csv', sep=',')

joined_false = imdb_df.join(
    movies.set_index('movieId'),
    how='left',
    on='movieId'
)
#display(joined_false.head(50))

#print('Число строк в таблице ratings_dates: ', imdb_dates.shape[0])
#print('Число строк в таблице merged: ', joined_false.shape[0])
#print(imdb_dates.shape[0] == joined_false.shape[0])

#merge_ratings = r_1.merge(r_2, how='outer')
#print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
#display(merge_ratings)



import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

joined_false['year_release'] = joined_false['title'].apply(get_year_release)
#display(joined_false.info())

mask7 = joined_false[(joined_false['year_release'] == 1999)]
bad_film = mask7.groupby('title')['rating'].agg('mean').sort_values()
#display(bad_film)
#movies_1999 = joined_false[mask7]['rating'].sort_values(ignore_index=True, ascending=True)
#print(movies_1999.iloc[0])
#print(joined_false.iloc[68328])

mask_genre = joined_false[(joined_false['year_release'] == 2010)]
bad_genres = mask_genre.groupby('genres')['rating'].agg('mean').sort_values()
#display(bad_genres)


cineguy = joined_false.groupby('userId')['genres'].agg('max').sort_values()
#display(joined_false.groupby('userId')['genres'].nunique().sort_values(ascending=False))

joined = joined_false

#display(joined.info())
fanatic = joined.groupby('userId')['rating'].agg(['count','mean']).sort_values(by=['count','mean'],ascending=[True,False])
#display(fanatic)

hotshit_mask = joined_false[(joined_false['year_release'] == 2018)]
hotshit = hotshit_mask.groupby('genres')['rating'].agg(['count','mean']).sort_values(by=['count','mean'],ascending=[False,True])
#display(hotshit)
joined['year_rating'] = joined['date'].dt.year
#print(joined['year_rating'])

y_pivot = joined.pivot_table(
    values='rating',
    index='year_rating',
    columns='genres',
    aggfunc='mean',
    fill_value=0
)
display(y_pivot['Animation|Children|Mystery'])

In [ ]:
import pandas as pd
import datetime as dt
orders = pd.read_csv('data/orders.csv', sep=';')
orders.info()
products = pd.read_csv('data/products.csv', sep=';')
products.info()

joined = orders.join(
    products.set_index('Product_ID'),
    how='left',
    on='ID товара'
)
display(joined.info())

whale = joined['ID Покупателя']
joined['profits'] = joined['Количество'] * joined['Price']
#display(joined)
joined[joined['Оплачен'] == 'Да'].groupby('ID Покупателя')['profits'].sum().sort_values(ascending=False)